In [1]:
import math
import numpy as np
import h5py
# import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import json
import random
from collections import OrderedDict
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# %matplotlib inline
# np.random.seed(0)

In [2]:
import cv2

In [3]:
EPOCHS = 10
BATCH_SIZE = 32
IMG_SIZE = 224
data_file = "dataset.txt"

In [4]:
weight_file ="vgg16_weights.npz"
weights = np.load(weight_file)

In [5]:
keys = sorted(weights.keys())
# for index, key in enumerate(keys):
#     print(index, key, np.shape(weights[key]))

In [6]:
def conv_layer(input_data,filter_size,stride_size,pad="SAME",name="conv",weight_name="W1",bias_name="b1"):
    print("creating layer :"+name)
    with tf.name_scope(name):
        W = tf.Variable(weights[name+"_W"],name="weight")
        b = tf.Variable(weights[name+"_b"],name="bias")
        
        conv = tf.nn.conv2d(input_data, W, strides=stride_size, padding=pad)
        out = tf.nn.bias_add(conv, b)
        act = tf.nn.relu(out)
        tf.summary.histogram("weights", W)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [7]:
def vggnet_v1(input_data):
    print("creating model")
    output=[]
    layer_input = input_data
    
    with tf.name_scope('preprocess') as scope:
        mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
        layer_input = layer_input-mean
            
    print("reading vgg16 json file")
    vgg = json.load(open("vgg16.json"),object_pairs_hook=OrderedDict)
       
    for index,layer in enumerate(vgg):
        if "conv" in layer:
            output = conv_layer(layer_input,vgg[layer]["weights"],vgg[layer]["stride"],vgg[layer]["pad"],layer,"weight"+str(index),"bias"+str(index))
        elif "pool" in layer:
            output = tf.nn.max_pool(layer_input, ksize=vgg[layer]["ksize"], strides=vgg[layer]["strides"], padding=vgg[layer]["padding"],name=layer)
        layer_input = output
    
    pool5_flat = tf.contrib.layers.flatten(output)

    print("creating layer : fc6")
    with tf.name_scope('fc6'):
        fc6_W = tf.Variable(weights["fc6_W"],name="fc6_W")
        fc6_b= tf.Variable(weights["fc6_b"],name="fc6_b")

        fc6 = tf.nn.bias_add(tf.matmul(pool5_flat, fc6_W), fc6_b)
        fc6 = tf.nn.relu(fc6)
    
#     fc6 = tf.contrib.layers.fully_connected(P3,4096, weights_initializer=fc6_W, biases_initializer=fc6_b, scope="fc6")
    tf.summary.histogram("fc6/relu", fc6)
    
    dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)

    print("creating layer : fc7")
    with tf.name_scope('fc7'):
        fc7_W = tf.Variable(weights["fc7_W"],name="fc7_W")
        fc7_b= tf.Variable(weights["fc7_b"],name="fc7_b")

        fc7 = tf.nn.bias_add(tf.matmul(dropout1, fc7_W), fc7_b)
        fc7 = tf.nn.relu(fc7)
    
#     fc7 = tf.contrib.layers.fully_connected(dropout1,4096, weights_initializer=fc7_W, biases_initializer=fc7_b, scope="fc7")
    tf.summary.histogram("fc7/relu", fc7)
    
    dropout2 = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    print("creating layer : fc8")
    fc8 = tf.contrib.layers.fully_connected(dropout2,2, biases_initializer=tf.constant_initializer(1.0), activation_fn=None, scope="fc8")
    tf.summary.histogram("fc8", fc8)
    
    
    print("model completed.")
    ##NEED FULLY CONNECTED LAYER
    return fc8

In [8]:
mean = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
print(mean)

[[[[ 123.68   116.779  103.939]]]]


In [9]:
def prepare_data(data):
    newData=[]
    labels=[]
    print("preparing data....")
    for sample in data:
        img_path,label = sample.strip().split(" ")
        img = cv2.imread(img_path)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        #img = Image.open(img_path)
        #img = img.resize((IMG_SIZE,IMG_SIZE))
        img = np.array(img)
        #img = img/255
        newData.append(img)
        if "cat" in img_path:
            labels.append(np.array([1,0]))
        elif "dog" in img_path:
            labels.append(np.array([0,1]))
    print("preparing data completed.")
    return np.array(newData),np.array(labels)

In [10]:
f = open(data_file,"r")
data = f.readlines()
random.shuffle(data)
print(len(data))
train_data = data[:20000]
train_X, train_Y = prepare_data(train_data)
val_data = data[20000:]
val_X, val_Y = prepare_data(val_data)

25000
preparing data....
preparing data completed.
preparing data....
preparing data completed.


In [11]:
# train_X = train_X - mean
# val_X = val_X - mean

In [12]:
# train_X.shape

In [14]:
tf.reset_default_graph()

start_learning_rate = 0.01

batch = tf.Variable(0, trainable=False)
train_size = 20000
val_size = 5000

learning_rate = tf.train.exponential_decay(
  0.001,                # Base learning rate.
  batch * BATCH_SIZE,  # Current index into the dataset.
  train_size,          # Decay step.
  0.1,                # Decay rate.
  staircase=True)

with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,IMG_SIZE,IMG_SIZE , 3),name="X")
    tf.summary.image('input', X, BATCH_SIZE)
    Y = tf.placeholder(tf.float32,shape=(None,2),name="labels")
    
logits = vggnet_v1(X)

with tf.name_scope("loss"):    
    train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", train_loss)

with tf.name_scope("train"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_loss,global_step=batch)
print("optimizer set")

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
print("accuracy set")

val_loss = tf.Variable(0.0)
tf.summary.scalar("loss", val_loss)

val_accuracy = tf.Variable(0.0)
tf.summary.scalar("accuracy", val_accuracy)


merged_summary = tf.summary.merge_all()


with tf.Session() as sess:
    print("session created.")
    print("initializing global variables")
    sess.run(tf.global_variables_initializer())
    
    print("writing tensorboard")
    train_writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg-train",sess.graph)
    val_writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg-val",sess.graph)

    
    print("writer added to graph")
    
    for epoch in range(EPOCHS):
        
        print("processing epoch "+str(epoch))
        print("total number of batches in 1 epoch: "+ str(len(train_data)/BATCH_SIZE))
        
        for i in range(0,train_size,BATCH_SIZE): 
            print("i: "+str(i))
            print("Iteration:" + str((i/BATCH_SIZE)+1))
            
            #minibatch_X, minibatch_Y = prepare_data(train_data[i:i+BATCH_SIZE])
            #sess.run(optimizer,feed_dict={X:minibatch_X,Y:minibatch_Y})
            
            minibatch_X = train_X[i : i + BATCH_SIZE]
            minibatch_Y = train_Y[i : i + BATCH_SIZE]
            
            _ , temp_loss, temp_accuracy = sess.run([optimizer,train_loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
            
            if i % (BATCH_SIZE*10) == 0 and i >200: 
                summary = sess.run(merged_summary,feed_dict={X:minibatch_X,Y:minibatch_Y})
                train_writer.add_summary(summary, i + epoch*len(train_data))
            
            
            
            print("temp_loss: "+str(temp_loss))
            
            if i % (BATCH_SIZE*50) == 0 and i>=1000:
                print("validating...")
                total_loss = 0
                total_accuracy = 0
                val_batch_size = 50
                val_batches = val_size/val_batch_size
                for v in range(0,val_size,val_batch_size):
                    #val_x,val_y = prepare_data(val_data[v:v+BATCH_SIZE])
                    
                    val_minibatch_x = val_X[v : v + val_batch_size]
                    val_minibatch_y = val_Y[v : v + val_batch_size]
                    temp_loss, temp_accuracy = sess.run([train_loss,accuracy],feed_dict={X:val_minibatch_x,Y:val_minibatch_y})
                    total_accuracy+=temp_accuracy
                    total_loss+=temp_loss
                
                val_loss = total_loss/val_batches
                print("val_loss: ")
                print(val_loss)
                
                val_accuracy = total_accuracy/val_batches
                print("val_accuracy: ")
                print(val_accuracy)
                
                val_summary = sess.run(merged_summary)
                val_writer.add_summary(val_summary, i + epoch*len(train_data))

                
                

#     train_accuracy = accuracy.eval({X: train_data, Y: train_labels})
#     val_data, val_labels = prepare_data(val_data)
#     val_accuracy = accuracy.eval({X: val_data, Y: val_labels})
# #     print("Train Accuracy:", train_accuracy)
#     print("Validation Accuracy:", val_accuracy)
    


creating model
reading vgg16 json file
creating layer :conv1_1
creating layer :conv1_2
creating layer :conv2_1
creating layer :conv2_2
creating layer :conv3_1
creating layer :conv3_2
creating layer :conv3_3
creating layer :conv4_1
creating layer :conv4_2
creating layer :conv4_3
creating layer :conv5_1
creating layer :conv5_2
creating layer :conv5_3
creating layer : fc6
creating layer : fc7
creating layer : fc8
model completed.
optimizer set
accuracy set
session created.
initializing global variables
writing tensorboard
writer added to graph
processing epoch 0
total number of batches in 1 epoch: 625.0
i: 0
Iteration:1.0
temp_loss: 2.33953
i: 32
Iteration:2.0
temp_loss: 127.545
i: 64
Iteration:3.0
temp_loss: 294.963
i: 96
Iteration:4.0
temp_loss: 15.9853
i: 128
Iteration:5.0
temp_loss: 480.616
i: 160
Iteration:6.0
temp_loss: 5.60068
i: 192
Iteration:7.0
temp_loss: 1.83533
i: 224
Iteration:8.0
temp_loss: 13.0807
i: 256
Iteration:9.0
temp_loss: 2.59038
i: 288
Iteration:10.0
temp_loss: 3.27

InvalidArgumentError: You must feed a value for placeholder tensor 'inputs/X' with dtype float and shape [?,224,224,3]
	 [[Node: inputs/X = Placeholder[dtype=DT_FLOAT, shape=[?,224,224,3], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: inputs/X/_245 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_280_inputs/X", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'inputs/X', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    a